<a href="https://colab.research.google.com/github/ltdaovn/dataset/blob/master/TheGioiDiDong/thegioididong_crawling_28_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install html5lib
!pip install lxml

!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# import

In [ ]:
import sys
from selenium import webdriver
import time

import bs4
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json

# setting

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
SESSION = requests.Session()
HEADERS = {
    'user-agent': ('Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/48.0.2564.109 Safari/537.36')
}

BASE_URL = 'https://www.thegioididong.com/'

# function

In [ ]:
def crawling(url,time_sleeping = 10):
    print('- webdriver is starting ... ')
    wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    wd.get(url)
    time.sleep(time_sleeping)
    page = wd.page_source
    soup = BeautifulSoup(page, 'html.parser')
    wd.close()

    return soup

In [ ]:
# lấy link những item xuất hiện trên một trang đã được tải đầy đủ theo type và end_page
def get_all_item_in_page(type,end_page):

    url = BASE_URL + type + str(end_page)
    print(url)
    soup = crawling(url)

    all_item = soup.find_all('li', {'class': ['item','ajaxed',' item ']})

    if len(all_item) > 0:
        print('- Found {number:} items'.format(number = len(all_item)))
        # href_all_item_in_page = [a.find('a').get('href') for a in all_item]
        href_all_item_in_page = []

        for a in all_item:
            try:
                h = a.find('a').get('href')
                href_all_item_in_page.append(h)
            except:
                print(a)

        if len(href_all_item_in_page) > 0:
            print('- Found links of {number:} items'.format(number = len(href_all_item_in_page)))
            return href_all_item_in_page
        else:
            print('- Not find the link of the item')
            return None
    else:
        print('- No item found')
        return None

In [ ]:
#get_all_item_in_page('dtdd#c=42&o=9&pi=',10)

In [ ]:
def rating_processing(rating):
    num_star = 0
    comment_star = rating.find('div',{'class':['comment-star']})
    for star in comment_star.find_all('i'):
        if star['class'][0] == 'icon-star':
            num_star += 1

    cmt_txt = rating.find('p',{'class':['cmt-txt']})
    comment = cmt_txt.text

    txtname = rating.find('p',{'class':['txtname']})
    username = txtname.text

    rating = {
        'username': username,
        'comment': comment,
        'star': num_star,
        }
    #print(rating)
    return rating

In [ ]:
def get_rating(rating_page_url,session, headers, except_loop = 8,num_page = 1):

    try:
        url = rating_page_url+'?p='+str(num_page)
        response = session.get(url,headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        except_loop -= 1
        if except_loop > 0:
            session = requests.Session() #requests lại
            get_rating(rating_page_url, session, headers ,except_loop = except_loop,num_page = num_page)
        else:
            raise
    rating_data = []
    rating_list = soup.find_all('div',{'class':['comment__item par']})
    if len(rating_list) > 0:
        for rating in rating_list:
            rating_data.append(rating_processing(rating))

    return rating_data

#rating_page_url = 'https://www.thegioididong.com/dtdd/realme-c12/danh-gia'
#rating_list = get_rating(rating_page_url , SESSION, HEADERS, except_loop = 8,num_page = 1)
#len(rating_list)

In [ ]:
def crawling_rating_data():
    print("main runing ...")

    categories = [('dtdd#c=42&o=9&pi=',14),('laptop#c=44&o=9&pi=',8),('may-tinh-bang#c=522&o=9&pi=',2)]

    link_all_items = []
    for type, end_page in categories:
        link_all_items[len(link_all_items):] = get_all_item_in_page(type,end_page)

    print('- Found {number:} items for all'.format(number = len(link_all_items)))


    data = []
    for link_item in link_all_items:
        if '?src=osp' in link_item:
            link_item = link_item.replace('?src=osp', "")

        rating_page_url = 'https://www.thegioididong.com'+link_item+'/danh-gia'
        print(rating_page_url)

        t = True
        n = 1
        while t:
            rating_data = get_rating(rating_page_url,SESSION, HEADERS, except_loop = 8, num_page = n)
            if rating_data == []:
                t = False
            else:
                data.append(rating_data)
                n += 1

    
    return data

# crawling

In [ ]:
rating_data = crawling_rating_data()
# thegioididong_rating_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/kkdl/thegioididong_rating_data_12_06_21.json'
# with open(thegioididong_rating_data_path, 'w', encoding='utf-8') as file: # file nằm ở thư mục cá nhân
#     json.dump(rating_data, file,ensure_ascii=False)

In [ ]:
from datetime import datetime
#print(datetime.today().strftime('%Y%m%d'))
thegioididong_rating_data_path = '/content/drive/MyDrive/Colab Notebooks/thegioididong_rating_data_' + datetime.today().strftime('%Y%m%d') + '.json'
with open(thegioididong_rating_data_path, 'w', encoding='utf-8') as file: # file nằm ở thư mục cá nhân
    json.dump(rating_data, file,ensure_ascii=False)